In [32]:
import os 
import numpy as np

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.python.client import device_lib
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from keras import backend as K 

from IAdeforestation.preprocessing import *
from IAdeforestation.training import *

In [2]:
print(device_lib.list_local_devices())

K.clear_session()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2319149645807300402
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3172404430
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6612914583601935007
physical_device_desc: "device: 0, name: GeForce GTX 960, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


In [11]:
SAVED_MODELS_PATH = os.path.join('saved_models', 'fine_tuning')
IMAGES_DIR = 'images' # 'D:\img32' # 'D:\\fall_images'
BASE_MODEL_NAME = 'trans_learning_4'

In [25]:
model = load_model(os.path.join('saved_models', 'transfer_learning', 'Coffee',BASE_MODEL_NAME, BASE_MODEL_NAME + '.h5'), compile=False)
model.trainable = True
model.training = False

In [33]:
PATH_COFFEE = os.path.join(IMAGES_DIR, '2')
path1 = [os.path.join(PATH_COFFEE, s) for s in os.listdir(PATH_COFFEE)]
class1 = [0]*len(path1)

other_class_dir = list(range(1,32))
other_class_dir.remove(2)
path2 = []

for p in other_class_dir:
    for name in os.listdir(os.path.join(IMAGES_DIR, str(p))):
        path2.append(os.path.join(IMAGES_DIR, str(p), name))
        
path2 = random.sample(path2, len(path1)) # Take the same number of img as class 0
    
class2 = [1]*len(path2)

file_paths = np.asarray(path1 + path2)
classes = np.asarray(class1 + class2)
path_train, path_test, y_train, y_test = train_test_split(file_paths, classes, test_size=0.20)

In [28]:
BATCH_SIZE = 64
LEARNING_RATE = 0.00001
DATA_AUGMENTATION_MULTIPLIATOR = 3
EPOCH = 100
RUN_NAME = 'finetuning_1'

In [29]:
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
train_generator = generator(path_train, 
                            y_train, 
                            vietnam_params['mean'], 
                            vietnam_params['std'], 
                            batch_size=BATCH_SIZE, 
                            is_data_augmentation=True)

test_generator = generator(path_test, 
                           y_test, 
                           vietnam_params['mean'], 
                           vietnam_params['std'], 
                           batch_size=BATCH_SIZE, 
                           is_data_augmentation=True)

In [ ]:
earlystopper = EarlyStopping(monitor='accuracy',patience=30,restore_best_weights=True)
tensorboard = TensorBoard(log_dir=os.path.join('logs', RUN_NAME),histogram_freq=1)

In [ ]:
history = model.fit(train_generator,
                        epochs=EPOCH,
                        steps_per_epoch=int(len(path_train)/BATCH_SIZE)*DATA_AUGMENTATION_MULTIPLIATOR,
                        validation_data=test_generator,
                        validation_steps=int(len(path_test)/BATCH_SIZE)*DATA_AUGMENTATION_MULTIPLIATOR,
                        callbacks=[earlystopper, tensorboard])